# Building permits by neighborhood

In this demo, we'll walk through the process of downloading and processing a dataset, using the concepts we learned today:
- `try` and `except`
- `time.sleep()`
- `dateutil.parser`
- `datetime.datetime`
- writing your own functions

In [24]:
import requests
import json
# define the endpoint for the api request
bp_api_endpoint = "https://data.seattle.gov/resource/mags-97de.json?"

In [25]:
#set parameters that we are passing to the API, to filter the data we want
bp_api_params = {
"$select" : "application_permit_number, action_type, address, category, value, issue_date, latitude, longitude",
"$where" : "(action_type='NEW' OR action_type='ADD/ALT') AND category='MULTIFAMILY' AND (status='Permit Issued' OR status='Permit Finaled' OR status='Permit Closed')",
"$order" : "issue_date DESC",
"$limit" : "10000",
}

In [26]:
#request the data from data.seattle.gov API and store it as a Python dictionary
bp_api_req = requests.get(bp_api_endpoint, params=bp_api_params)
bp_api_data = bp_api_req.json()
print(bp_api_data[0])
print(len(bp_api_data))

{'longitude': '-122.35414135', 'value': '311000', 'application_permit_number': '6118877', 'action_type': 'NEW', 'address': '8725 PHINNEY AVE N', 'category': 'MULTIFAMILY', 'latitude': '47.69321735'}
2744


In [27]:
#save the JSON object as a file as well, just in case we lose access to the API later
with open('multifamily_permits_2010-2016.json', 'w') as json_outfile:
    json.dump(bp_api_data, json_outfile)

In [28]:
def queryGmapsAPI(query_params, query_type):
    if query_type == "latlng":
        gmaps_api_params = {'latlng' : query_params}
    elif query_type == "address":
        gmaps_api_params = {'address' : query_params}
    else:
        return "unrecognized query type. Please use 'latlng' or 'address'"
#     q_string = "http://maps.googleapis.com/maps/api/geocode/json?" + query_type + "=" + query_params
#     print(q_string)
    gmaps_api_req = requests.get("http://maps.googleapis.com/maps/api/geocode/json", params=gmaps_api_params)
    gmaps_api_data = gmaps_api_req.json()

    return gmaps_api_data

In [ ]:
import time
with open("multifamily_permits_2010-2016.json") as json_infile:
    bp_api_data = json.load(json_infile)
# print(json.dumps(bp_api_data, indent=2))

In [32]:
counter = 0
permits_with_neighborhoods = 0
permits_by_neighborhood = {}
skipped_records = []

for x in bp_api_data:
    if counter < 10:
        print(counter)
        counter+= 1

        try:
            loc_params = x['latitude'] + "," + x["longitude"]
            loc_type = "latlng"
            loc_data = queryGmapsAPI(loc_params, loc_type)
        except KeyError:
            print("Couldn't find lat/lng for %s, trying query again with address" % (x['application_permit_number']))
            loc_params = x['address'] + "Seattle WA"
            loc_type = "address"            
            loc_data = queryGmapsAPI(loc_params, loc_type)

        if loc_data['status']:
            if loc_data['status'] == "OK":
                neighborhood = loc_data['results'][0]['address_components'][2]['long_name']
                permits_with_neighborhoods += 1        
                print("Found neighborhood for %s: %s" % (x['application_permit_number'], neighborhood))
                time.sleep(0.2)
            elif loc_data['status'] == "OVER_QUERY_LIMIT":                
                print("Could not get neighborhood data for record %s: %s" 
                      % (x['application_permit_number'], loc_data['status']))
                print(loc_data)
                skipped_records.append(x)
                time.sleep(0.2)
                continue
            elif loc_data['status'] == "ZERO_RESULTS":
                print("unrecognized location %s for record %s" 
                      % (x['address'], x['application_permit_number']))
                skipped_records.append(x)
                time.sleep(0.2)  
                continue
            else:
                print("unrecognized status in API response: %s" 
                      % (loc_data['status']))
                skipped_records.append(x)
                time.sleep(0.2)
                continue
        else:
            print(loc_data)
            continue

        if neighborhood in permits_by_neighborhood:
            permits_by_neighborhood[neighborhood] += 1
        else:
            permits_by_neighborhood[neighborhood] = 1
    else:
        break  

print("%d records were analyzed" % (counter))
print("/n")
print("Neighborhoods were identified for %d records" % (permits_with_neighborhoods))
print("/n")
print("%d records were skipped" % (len(skipped_records)))
print("/n")
print("Here's your data!")
print(json.dumps(permits_by_neighborhood, indent = 2))


0
Found neighborhood for 6118877: Greenwood
1
Could not get neighborhood data for record 6118890: OVER_QUERY_LIMIT
{'status': 'OVER_QUERY_LIMIT', 'error_message': 'You have exceeded your daily request quota for this API. We recommend registering for a key at the Google Developers Console: https://console.developers.google.com/apis/credentials?project=_', 'results': []}
2
Could not get neighborhood data for record 6526311: OVER_QUERY_LIMIT
{'status': 'OVER_QUERY_LIMIT', 'error_message': 'You have exceeded your daily request quota for this API. We recommend registering for a key at the Google Developers Console: https://console.developers.google.com/apis/credentials?project=_', 'results': []}
3
Found neighborhood for 6494431: Capitol Hill
4
Could not get neighborhood data for record 6533460: OVER_QUERY_LIMIT
{'status': 'OVER_QUERY_LIMIT', 'error_message': 'You have exceeded your daily request quota for this API. We recommend registering for a key at the Google Developers Console: https:/

In [5]:
with open("multifamily_permits_2010-2016.json") as json_infile:
    bp_api_data = json.load(json_infile)

print(json.dumps(bp_api_data, indent=2))

[
  {
    "longitude": "-122.37549243",
    "value": "104821",
    "issue_date": "2016-05-13T00:00:00",
    "application_permit_number": "6526311",
    "action_type": "ADD/ALT",
    "latitude": "47.63886596",
    "category": "MULTIFAMILY",
    "address": "1404 W BOSTON ST"
  },
  {
    "longitude": "-122.26612732",
    "value": "75000",
    "issue_date": "2016-05-12T00:00:00",
    "application_permit_number": "6533460",
    "action_type": "ADD/ALT",
    "latitude": "47.67450527",
    "category": "MULTIFAMILY",
    "address": "6323 SAND POINT WAY NE"
  },
  {
    "longitude": "-122.26612732",
    "value": "75000",
    "issue_date": "2016-05-12T00:00:00",
    "application_permit_number": "6533460",
    "action_type": "ADD/ALT",
    "latitude": "47.67450527",
    "category": "MULTIFAMILY",
    "address": "6323 SAND POINT WAY NE"
  },
  {
    "longitude": "-122.30442270",
    "value": "36306",
    "issue_date": "2016-05-12T00:00:00",
    "application_permit_number": "6517579",
    "action_

In [77]:
#save the JSON object as a file as well, just in case we lose access to the API later
with open('permits_by_neighborhood_2010-2016_1228.json', 'w') as json_outfile:
    json.dump(permits_by_neighborhood, json_outfile)

In [9]:
from dateutil import parser
for x in bp_api_data:
    x['issue_datetime'] = parser.parse(x['issue_date'])

print(bp_api_data[875])
print(bp_api_data[875]['issue_datetime'].year)


{'longitude': '-122.32756652', 'value': '20000', 'issue_date': '2013-10-08T00:00:00', 'application_permit_number': '6386469', 'issue_datetime': datetime.datetime(2013, 10, 8, 0, 0), 'action_type': 'ADD/ALT', 'latitude': '47.62332368', 'category': 'MULTIFAMILY', 'address': '308 E REPUBLICAN ST'}
2013


In [22]:
import datetime
oldest_record = datetime.datetime.today()

for x in bp_api_data:
    if x['issue_datetime'] < oldest_record:
        oldest_record = x['issue_datetime']

print("The oldest record in our dataset is from %s" % (oldest_record.strftime("%m/%d/%Y")))

newest_record = oldest_record
for x in bp_api_data:
    if x['issue_datetime'] > newest_record:
        newest_record = x['issue_datetime']

print("The newest record in our dataset is from %s" % (newest_record.strftime("%m/%d/%Y")))


The oldest record in our dataset is from 05/19/2011
The newest record in our dataset is from 05/13/2016
